In [31]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [32]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
select id_dmc, circuit_nr, 
max(assigment) as assigment, 
max(flow) as flow, 
max(ref_time) as ref_time,
max(set_point) as set_point,
max(start_delay) as start_delay,
max(temp) as temp,
max(working_mode) as working_mode,
max(timestamp) as timestamp
from z3dmc.oni_circuits
group by id_dmc, circuit_nr
order by id_dmc
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [34]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [35]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']


In [36]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
print("\n ONI_CIRCUITS after drop of columns: \n")
print(data['ONI_CIRCUITS'].head())


 ONI_CIRCUITS after drop of columns: 

   id_dmc  circuit_nr  assigment  flow  ref_time  set_point  start_delay  \
0  235342           1          0   0.0       0.0        0.0          0.0   
1  235342           2          0   0.0       0.0        0.0          0.0   
2  235342           3          0   0.0       0.0        0.0          0.0   
3  235342           4          0   0.0       0.0        0.0          0.0   
4  235342           5          0   0.0       0.0        0.0          0.0   

   temp  working_mode  
0   0.0             0  
1   0.0             0  
2   0.0             0  
3   0.0             0  
4   0.0             0  


In [37]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
print("\n MEB_DGM after drop of columns: \n")
print(data['MEB_DGM'].head(5))


 MEB_DGM after drop of columns: 

     id                    dmc  nr_dgm status  czas_fazy_1  czas_fazy_2  \
0   220  190627091611011003610    10.0      4       1850.0         88.0   
1  3416  190719071611011006077    10.0      1       2197.0         89.0   
2  3417  190719071811011006078    10.0      1       2184.0         86.0   
3  3418  190719072011011006079    10.0      1       2159.0         89.0   
4  3419  190719072111011006080    10.0      1       2175.0         89.0   

   czas_fazy_3  max_predkosc  cisnienie_tloka  cisnienie_koncowe  ...  \
0         47.5          6.52              7.0              264.0  ...   
1         46.5          6.59             11.0              285.0  ...   
2         45.0          6.52              8.0              284.0  ...   
3         45.5          6.54              6.0              284.0  ...   
4         47.0          6.57              8.0              283.0  ...   

   oni_temp_curr_f2  oni_temp_fore_f1  oni_temp_fore_f2  vds_air_pressure  

In [38]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
print("\n MEB_DMC after drop of columns: \n")
print(data['MEB_DMC'].head(5))


 MEB_DMC after drop of columns: 

   id_dmc                                         dmc  status_koncowy  \
0    2063  0MH301103CVW41910090436321102S111910040455             2.0   
1    2064  0MH301103CVW41910090436411102S111910040410             1.0   
2    2065  0MH301103CVW41910090436521102S111910040350             2.0   
3    2066  0MH301103CVW41910090436611102S111910040356             2.0   
4    2067  0MH301103CVW41910090436721102S111910040345             2.0   

  dmc_casting  
0        None  
1        None  
2        None  
3        None  
4        None  


In [39]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
print("\n MEB_KO after drop of columns: \n")
print(data['MEB_KO'].head(5))


 MEB_KO after drop of columns: 

   id_dmc  nok_strefa  nok_rodzaj  status_ko  rn
0    2007           0           0          0   1
1    2062           0           0          1   1
2    2063         511         201          2   1
3    2064           0           0          1   1
4    2065         505         201          2   1


In [41]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
print("\n MEB_KO_DGM after drop of columns: \n")
print(data['MEB_KO_DGM'].head(5))


 MEB_KO_DGM after drop of columns: 

   id_dmc  nok_strefa  nok_rodzaj  status_ko  rodzaj_kontroli product_id  \
0   67460           0           0          1                1       None   
1   67475           0           0          1                1       None   
2   67521           0           0          1                1       None   
3   67522           0           0          1                1       None   
4   67523           0           0          1                1       None   

  line_id  rn  
0    None   1  
1    None   1  
2    None   1  
3    None   1  
4    None   1  


In [40]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
print("\n 'MEB_KS' after drop of columns: \n")
print(data['MEB_KS'].head(5))


 'MEB_KS' after drop of columns: 

   cisnienie  przeciek  nrprogramu  temperaturatestu  statusszczelnosc  \
0      1.051     0.398           1         23.680000                 1   
1      1.054     0.393           1         23.719999                 1   
2      1.053     0.794           1         22.519999                 1   
3      1.055     0.433           1         22.600000                 1   
4      0.992    24.202           1         22.840000                 3   

   statusdmc  id_dmc  rn  
0        1.0    1917   1  
1        1.0    1918   1  
2        1.0    1919   1  
3        1.0    1920   1  
4        1.0    1921   1  


In [42]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
print("\n 'MEB_GROB' after drop of columns: \n")
print(data['MEB_GROB'].head(5))



 'MEB_GROB' after drop of columns: 

   id_meb_grob  id_dmc  part_type  part_status  last_machine_number  \
0            5    5746          1            1                    1   
1            6    5753          1            1                    1   
2            7    5754          1            1                    1   
3            8    5756          1            1                    1   
4            9    5758          1            1                    1   

   last_pcf_number  temp_workpiece  temp_hydraulics  pressure_pcf_1  \
0                2       23.400000        33.400002           100.0   
1                1       23.500000        33.400002           100.0   
2                2       23.299999        33.400002           100.0   
3                1       23.500000        33.400002           100.0   
4                2       23.299999        33.400002           100.0   

   pressure_pcf_2  pressure_pcf_3  machine_nr  rn  
0           100.0            89.0         1.0   1  
1   

<h1>Łączenie tabel</h1>

In [43]:
#pd.set_option('display.max_columns', None)

# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [44]:
# łączę tabelę MEB_DMC z tabelą MEB_KO

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')

#  do wyrzucenia kolumna timestamp oraz eks

#data['MEB_DMC'].head(1)

In [45]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM

data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
#data['MEB_DGM'].head(1)

In [46]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'working_mode'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM

oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
#oni_circuits.head(1)

In [47]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS
data['MEB_DMC'] = data['MEB_DMC'].merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left')
#data['MEB_DMC'].head(1)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
#data['MEB_DMC'].head(1)

In [ ]:
pd.set_option('display.max_rows', None)
data['MEB_DMC'].columns.tolist()

['id_dmc',
 'dmc_x',
 'status_koncowy',
 'dmc_casting',
 'nok_strefa_x',
 'nok_rodzaj_x',
 'status_ko_x',
 'rn_x',
 'kod_pola_x',
 'rodzaj_uszkodzenia_x',
 'id_dmc_x',
 'flow_1',
 'flow_2',
 'flow_3',
 'flow_4',
 'flow_5',
 'flow_6',
 'flow_7',
 'flow_8',
 'flow_9',
 'flow_10',
 'flow_11',
 'flow_12',
 'flow_13',
 'flow_14',
 'flow_15',
 'flow_16',
 'flow_17',
 'flow_18',
 'flow_19',
 'flow_20',
 'flow_21',
 'flow_22',
 'flow_23',
 'flow_24',
 'flow_25',
 'flow_26',
 'flow_27',
 'flow_28',
 'temp_1',
 'temp_2',
 'temp_3',
 'temp_4',
 'temp_5',
 'temp_6',
 'temp_7',
 'temp_8',
 'temp_9',
 'temp_10',
 'temp_11',
 'temp_12',
 'temp_13',
 'temp_14',
 'temp_15',
 'temp_16',
 'temp_17',
 'temp_18',
 'temp_19',
 'temp_20',
 'temp_21',
 'temp_22',
 'temp_23',
 'temp_24',
 'temp_25',
 'temp_26',
 'temp_27',
 'temp_28',
 'id',
 'dmc_y',
 'nr_dgm',
 'status',
 'czas_fazy_1',
 'czas_fazy_2',
 'czas_fazy_3',
 'max_predkosc',
 'cisnienie_tloka',
 'cisnienie_koncowe',
 'nachdruck_hub',
 'anguss',
 'm

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS

data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
#data['MEB_DMC'].head(1)

MergeError: Passing 'suffixes' which cause duplicate columns {'rn_x'} is not allowed.

In [ ]:
"""connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()"""

"connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')\nconnection.version\nconnection.close()"

In [ ]:
"""try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)"""

"try:\n    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:\n        with connection.cursor() as cursor:\n            cursor.execute(dmc_sql)\n            while True:\n                row = cursor.fetchone()\n                if row is None:\n                    break\n                print(row)\n                data={'id':row[0],'dmc':row[1]}\n    cursor.close()\n    connection.close()\nexcept cx_Oracle.Error as error:\n    print(error)"

In [ ]:
#dodej to Adam przystojniaku

1